# 自动微分运算 — torch.autograd
- 译文：https://pytorch.apachecn.org/2.0/tutorials/beginner/basics/autogradqs_tutorial
- 原文：https://pytorch.org/tutorials/beginner/basics/autogradqs_tutorial.html

## 概览

- 在反向传播算法中，模型参数根据损失函数对参数的梯度进行更新。
- `torch.autograd` 是 PyTorch 的自动微分引擎，会为张量构建动态的计算图并自动计算梯度。
- 本笔记演示如何：设置 `requires_grad`、观察 `grad_fn`、调用 `backward()`、禁用梯度追踪，以及计算雅可比乘积（Jacobian-vector product）。

In [ ]:
import torch

# 简单单层网络示例，展示计算图与 grad_fn
x = torch.ones(5)                 # 输入 tensor
y = torch.zeros(3)                # 期望输出
w = torch.randn(5, 3, requires_grad=True)  # 需求梯度的参数
b = torch.randn(3, requires_grad=True)

z = torch.matmul(x, w) + b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

print(f"Gradient function for z = {z.grad_fn}")
print(f"Gradient function for loss = {loss.grad_fn}")

In [ ]:
# 计算梯度：对 loss 调用 backward，然后读取参数的 grad 属性
loss.backward()
print('w.grad:')
print(w.grad)
print('\nb.grad:')
print(b.grad)

# 注意：在实际训练循环中，通常会在每次迭代前把梯度清零：optimizer.zero_grad() 或 param.grad.zero_()

In [ ]:
# 禁用梯度追踪的两种方法：torch.no_grad() 和 detach()
z = torch.matmul(x, w) + b
print('z.requires_grad before no_grad:', z.requires_grad)

with torch.no_grad():
    z2 = torch.matmul(x, w) + b
    print('z2.requires_grad inside no_grad:', z2.requires_grad)

z_det = z.detach()
print('z_det.requires_grad after detach():', z_det.requires_grad)

## 计算图（DAG）与反向传播原理

- `autograd` 在前向传播时构建由 `Function` 对象组成的有向无环图（DAG），跟踪每次运算并保存反向传播所需的函数引用（保存在 `tensor.grad_fn`）。
- 调用 `loss.backward()` 会从根节点（loss）开始按链式法则向后计算梯度，并把结果累加到叶子节点（设置了 `requires_grad=True` 的张量）的 `.grad` 属性中。
- 注意：计算图是动态重建的；每次前向传播都会创建新的图。若对同一图多次调用 `backward()`，需使用 `retain_graph=True` 或在每次调用前清理/重建图。

In [ ]:
# 雅可比（Jacobian）乘积示例：对非标量输出调用 backward 时需传入向量参数
inp = torch.eye(4, 5, requires_grad=True)
out = (inp + 1).pow(2).t()
# 传入与 out 相同形状的张量作为 backward 的参数
out.backward(torch.ones_like(out), retain_graph=True)
print('First call\n', inp.grad)
# 第二次调用会累加梯度
out.backward(torch.ones_like(out), retain_graph=True)
print('\nSecond call\n', inp.grad)
# 清零再调用
inp.grad.zero_()
out.backward(torch.ones_like(out), retain_graph=True)
print('\nCall after zeroing gradients\n', inp.grad)

## 小结与参考

- 将模型参数设为 `requires_grad=True`，前向构建计算图，调用 `backward()` 自动计算梯度，并从参数的 `.grad` 中读取结果。
- 在评估或推理阶段使用 `torch.no_grad()` 或 `tensor.detach()` 禁用梯度追踪以加速计算并节省内存。
- 对非标量输出可以通过向 `backward()` 传入合适形状的向量来计算雅可比乘积。